In [108]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, \
  mean_absolute_error, r2_score, mean_squared_error
import matplotlib.pyplot as plt

In [109]:
df_returns = pd.DataFrame()

In [110]:
df_returns = pd.read_csv('S&P-returns.csv', index_col=0, parse_dates=True)
df = pd.read_csv('S&P100.csv', index_col=0, parse_dates=True)

#for name in df.columns:
#  df_returns[name] = np.log(df[name]).diff()

In [111]:
df.isna().sum().sum()

0

In [112]:
df_returns.head()

,AAPL,MSFT,AMZN,GOOGL,GOOG,PG,TSLA,BAC,NVDA,JNJ,UNH,JPM,OEF,SPY
Date,,,,,,,,,,,,,,
2012-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-04,0.005360,0.023261,-0.008526,0.004304,0.004304,-0.000449,-0.013264,0.001723,0.011332,-0.006090,0.014461,-0.000858,0.001721,0.001567
2012-01-05,0.011041,0.010167,0.000563,-0.013969,-0.013969,-0.004201,-0.021522,0.082555,0.035286,-0.001223,0.006677,0.020672,0.002061,0.002659
2012-01-06,0.010400,0.015415,0.027763,-0.013736,-0.013736,-0.002408,-0.007773,-0.020817,-0.011624,-0.008754,0.003606,-0.009009,-0.002577,-0.002581
2012-01-09,-0.001587,-0.013250,-0.022428,-0.043324,-0.043324,0.004211,0.012556,0.014458,0.000000,0.001541,-0.001137,-0.001698,0.001891,0.002424


In [113]:
df_returns['OEF'] = df_returns['OEF'].shift(-1)
df_returns['OEF'].tail()

Date
2022-01-19   -0.011317
2022-01-20   -0.021936
2022-01-21    0.002917
2022-01-24   -0.011268
2022-01-25         NaN
Name: OEF, dtype: float64

In [114]:
Ntest = 1000
train = df_returns.iloc[1:-Ntest]
test = df_returns.iloc[-Ntest:-1]

In [115]:
df_returns.columns

Index(['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'GOOG', 'PG', 'TSLA', 'BAC', 'NVDA',
       'JNJ', 'UNH', 'JPM', 'OEF', 'SPY'],
      dtype='object')

In [116]:
# x_cols = df.columns.drop('OEF')
x_cols = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 
         'GOOG', 'PG', 'TSLA', 'BAC', 
         'NVDA', 'JNJ', 'UNH', 'JPM']
x_cols

['AAPL',
 'MSFT',
 'AMZN',
 'GOOGL',
 'GOOG',
 'PG',
 'TSLA',
 'BAC',
 'NVDA',
 'JNJ',
 'UNH',
 'JPM']

In [117]:
Xtrain = train[x_cols]
Ytrain = train['OEF']
Xtest = test[x_cols]
Ytest = test['OEF']

In [118]:
Ytest.head()

Date
2018-02-06   -0.005442
2018-02-07   -0.038771
2018-02-08    0.014727
2018-02-09    0.014174
2018-02-12    0.002878
Name: OEF, dtype: float64

In [119]:
Xtrain.head()

,AAPL,MSFT,AMZN,GOOGL,GOOG,PG,TSLA,BAC,NVDA,JNJ,UNH,JPM
Date,,,,,,,,,,,,
2012-01-04,0.005360,0.023261,-0.008526,0.004304,0.004304,-0.000449,-0.013264,0.001723,0.011332,-0.006090,0.014461,-0.000858
2012-01-05,0.011041,0.010167,0.000563,-0.013969,-0.013969,-0.004201,-0.021522,0.082555,0.035286,-0.001223,0.006677,0.020672
2012-01-06,0.010400,0.015415,0.027763,-0.013736,-0.013736,-0.002408,-0.007773,-0.020817,-0.011624,-0.008754,0.003606,-0.009009
2012-01-09,-0.001587,-0.013250,-0.022428,-0.043324,-0.043324,0.004211,0.012556,0.014458,0.000000,0.001541,-0.001137,-0.001698
2012-01-10,0.003574,0.003598,0.004359,0.001092,0.001092,-0.004663,0.013487,0.055828,-0.004135,0.004150,-0.001899,0.021024


## Buy & Hold

In [120]:
#std buy-and-hold train and test
Ytrain.std(), Ytest.std()

(0.007614319076529266, 0.013529851199317393)

In [121]:
#SR buy-and-hold train and test
Ytrain.mean()/Ytrain.std(), Ytest.mean()/Ytest.std()

(0.06192600988326031, 0.038053469009501795)

In [122]:
# Total return buy-and-hold train and test
Ytrain.sum(), Ytest.sum()

(0.7223753783295717, 0.5143429155430788)

## XGBoost Regressor

In [123]:
from xgboost.sklearn import XGBRegressor
model = XGBRegressor()
model.fit(Xtrain, Ytrain)
model.score(Xtrain, Ytrain), model.score(Xtest, Ytest)

(0.984121113406096, -0.27870966579787937)

In [124]:
# Direction
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
np.mean(np.sign(Ptrain) == np.sign(Ytrain)), np.mean(np.sign(Ptest) == np.sign(Ytest))

(0.9556135770234987, 0.5105105105105106)

In [125]:
set(np.sign(Ptrain)), set(np.sign(Ptest))

({-1.0, 1.0}, {-1.0, 1.0})

In [126]:
train_idx = df.index <= train.index[-1]
test_idx = df.index > train.index[-1]

train_idx[0] = False
test_idx[-1] = False

In [127]:
df_returns['Position'] = 0 # create new column
df_returns.loc[train_idx,'Position'] = (Ptrain > 0)
df_returns.loc[test_idx,'Position'] = (Ptest > 0)

In [128]:
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['OEF']

In [129]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

4.470544875801163

In [130]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.11559217822581314

In [131]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.004572352331086935, 0.6382076491363692)

In [132]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.009704051703708022, 0.011923667520002169)

In [133]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(9.200228551268258e-07, 0.00023384228259967047)

In [134]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.0009591782186469967, 0.015291902517334803)

In [135]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.0006792364903412241, 0.009518913725953846)

In [136]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(0.984121113406096, -0.27870966579787937)

In [137]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(20983945435.583603, 67216154634.53976)

In [138]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

0.30983184419542936

In [139]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.532008934206025

In [140]:
df_returns['Position'].to_csv('XGBRBool.csv', index = True)
sum(df_returns['Position'])

1394

## LGBM Regressor

In [141]:
from lightgbm import LGBMRegressor
model = LGBMRegressor()
model.fit(Xtrain, Ytrain)
model.score(Xtrain, Ytrain), model.score(Xtest, Ytest)

(0.8024223076596085, -0.11617099655500462)

In [142]:
# Direction
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
np.mean(np.sign(Ptrain) == np.sign(Ytrain)), np.mean(np.sign(Ptest) == np.sign(Ytest))

(0.8753263707571801, 0.5015015015015015)

In [143]:
set(np.sign(Ptrain)), set(np.sign(Ptest))

({-1.0, 1.0}, {-1.0, 1.0})

In [144]:
train_idx = df.index <= train.index[-1]
test_idx = df.index > train.index[-1]

train_idx[0] = False
test_idx[-1] = False

In [145]:
df_returns['Position'] = 0 # create new column
df_returns.loc[train_idx,'Position'] = (Ptrain > 0)
df_returns.loc[test_idx,'Position'] = (Ptest > 0)

In [146]:
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['OEF']

In [147]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

4.247848247199831

In [148]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.11836055311913629

In [149]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.004661790604998658, 0.5947815190748617)

In [150]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.010085247819768486, 0.011747756155188593)

In [151]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(1.144765355816326e-05, 0.0002041182455933885)

In [152]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.0033834381268412848, 0.01428699568115664)

In [153]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.002419180296357521, 0.009019050912406146)

In [154]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(0.8024223076596085, -0.11617099655500462)

In [155]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(17974776069.33198, 51058719859.89038)

In [156]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

0.7592580262595109

In [157]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.5479624508043606

In [158]:
df_returns['Position'].to_csv('LGBMRBool.csv', index = True)
sum(df_returns['Position'])

1408

## XGBoost Classifier

In [159]:
from xgboost.sklearn import XGBClassifier
model = XGBClassifier()
Ctrain = (Ytrain > 0)
Ctest = (Ytest > 0)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

C:\Users\rapha\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:38:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


(1.0, 0.4974974974974975)

In [160]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['OEF']
set(Ptrain), set(Ptest)

({False, True}, {False, True})

In [161]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

4.498574828303063

In [162]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.10007186685410385

In [163]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.004559243472961057, 0.6440556560105448)

In [164]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.009228905168443605, 0.01085416277063016)

In [165]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(0.5353080646586249, 0.5845673792186874)

In [166]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.7316475002749787, 0.7645700616808687)

In [167]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.5406511910063123, 0.5882874667278121)

In [168]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(-9238.005318345939, -3195.5645810802707)

In [169]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(381.35145625155474, 18032430940827.61)

In [170]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

1.988278467528496

In [171]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.9906257939659986

In [172]:
df_returns['Position'].to_csv('XGBCBool.csv', index = True)
sum(df_returns['Position'])

1413

## LGBM Classifier

In [173]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()
Ctrain = (Ytrain > 0)
Ctest = (Ytest > 0)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

(0.9973890339425587, 0.4774774774774775)

In [174]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['OEF']
set(Ptrain), set(Ptest)

({False, True}, {False, True})

In [175]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

4.465993113570026

In [176]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.0109537217968656

In [177]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.004603604835978125, 0.6332296563102059)

In [178]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.010027113756346209, 0.0010935037489137236)

In [179]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(0.5379615655916967, 0.5887497979275108)

In [180]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.7334586325019952, 0.7673003309835796)

In [181]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.5432621570637535, 0.5920347819388714)

In [182]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(-9283.802702789455, -3218.4351208728626)

In [183]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(382.70719004917527, 13524323205711.727)

In [184]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

1.988278467528496

In [185]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.9901346802846238

In [186]:
df_returns['Position'].to_csv('LGBMCBool.csv', index = True)
sum(df_returns['Position'])

1421

## KNN

In [187]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
Ctrain = (Ytrain > 0)
Ctest = (Ytest > 0)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

(0.664490861618799, 0.5205205205205206)

In [188]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['OEF']
set(Ptrain), set(Ptest)

({False, True}, {False, True})

In [189]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

1.6883934722935372

In [190]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.2906411778454503

In [191]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.005299364440123782, 0.20796541253071638)

In [192]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.009219295235601036, 0.03155687094518336)

In [193]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(0.5611699202147731, 0.5491508240415275)

In [194]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.7491127553411255, 0.7410471132401283)

In [195]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.5646603660144639, 0.5531418051076117)

In [196]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(-9684.361046571183, -3001.8977603033422)

In [197]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(20577805086312.715, 22540538675866.914)

In [198]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

1.991935490279464

In [199]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.990449609272952

In [200]:
df_returns['Position'].to_csv('KNNBool.csv', index = True)
sum(df_returns['Position'])

1412

## Naive Bayes Gaussian Classifier

In [201]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
Ctrain = (Ytrain > 0)
Ctest = (Ytest > 0)
model.fit(Xtrain, Ctrain)
model.score(Xtrain, Ctrain), model.score(Xtest, Ctest)

(0.5600522193211488, 0.5385385385385385)

In [202]:
Ptrain = model.predict(Xtrain)
Ptest = model.predict(Xtest)
df_returns.loc[train_idx,'Position'] = Ptrain
df_returns.loc[test_idx,'Position'] = Ptest
df_returns['AlgoReturn'] = df_returns['Position'] * df_returns['OEF']
set(Ptrain), set(Ptest)

({False, True}, {False, True})

In [203]:
# Total algo log return train
df_returns.iloc[1:-Ntest]['AlgoReturn'].sum()

0.7570569666850071

In [204]:
# Total algo log return test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].sum()

0.7279172817414654

In [205]:
#std and SR algo train
df_returns.iloc[1:-Ntest]['AlgoReturn'].std(), df_returns.iloc[1:-Ntest]['AlgoReturn'].mean()/df_returns.iloc[1:-Ntest]['AlgoReturn'].std()

(0.006097252190686221, 0.08104675605280826)

In [206]:
#std and SR algo test
df_returns.iloc[-Ntest:-1]['AlgoReturn'].std(), df_returns.iloc[-Ntest:-1]['AlgoReturn'].mean()/df_returns.iloc[-Ntest:-1]['AlgoReturn'].std()

(0.010372186287960534, 0.07024998466474776)

In [207]:
# MSE
mean_squared_error(Ytrain, Ptrain), mean_squared_error(Ytest, Ptest)

(0.6263544326241837, 0.5943214424521461)

In [208]:
# RMSE
mean_squared_error(Ytrain, Ptrain, squared=False), mean_squared_error(Ytest, Ptest, squared=False)

(0.7914255698574464, 0.7709224620233517)

In [209]:
# MAE
mean_absolute_error(Ytrain, Ptrain), mean_absolute_error(Ytest, Ptest) 

(0.6287038112501566, 0.5981634648752836)

In [210]:
# R^2
r2_score(Ytrain, Ptrain), r2_score(Ytest, Ptest)

(-10809.395576376726, -3248.9023042617537)

In [211]:
# MAPE
mean_absolute_percentage_error(Ytrain, Ptrain), mean_absolute_percentage_error(Ytest, Ptest)

(14698432204645.775, 13524323205702.113)

In [212]:
#sMAPE train
def smape(Ytrain, Ptrain):
    numerator = np.abs(Ytrain - Ptrain)
    denominator = (np.abs(Ytrain) + np.abs(Ptrain)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytrain, Ptrain)

1.9920764795265442

In [213]:
#sMAPE test
def smape(Ytest, Ptest):
    numerator = np.abs(Ytest - Ptest)
    denominator = (np.abs(Ytest) + np.abs(Ptest)) / 2
    ratio = numerator / denominator
    return ratio.mean()

smape(Ytest, Ptest)

1.9884630236225973

In [214]:
df_returns['Position'].to_csv('NBGCBool.csv', index = True)
sum(df_returns['Position'])

1556